## Introduction
嗯这个Introduction吧，主要告诉你如何在不依靠Estimator的情况下，运行并管理Tensorflow程序，以及如何在这种较底层的环境使用高层组件(dataset,layers和feature columns)

我们(Tensorflow的程序员们)推荐尽肯能的使用高层API来搭建模型，但了解Tensorflow的核心部分仍然是有价值的：
* 如果你知道低层API如何工作的话，实验与debug过程都会更加直观
* 就算你使用高层API时，也了解模型内部工作机制

### 1.Setup
在此之前，请安装Tensorflow，且需要了解以下内容：

* 如何用Python编程
* 或者至少了解一点arrays
* 知道一点机器学习就更理想了

(个人觉得还需要点线性代数知识？)

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

### 2.Tensor Values
Tensor是Tensorflow中储存数据的核心，一个tensor包含一系列包装在任何维度下的数列(Tensor的阶就是所说的维度了)，而tensor的shape，表示为包装在元组中的几个整数，特指在每个维度下的数列的长度。

一些示例如下👇

In [ ]:
3. # a rank 0 tensor; a scalar with shape [],
[1., 2., 3.] # a rank 1 tensor; a vector with shape [3]
[[1., 2., 3.], [4., 5., 6.]] # a rank 2 tensor; a matrix with shape [2, 3]
[[[1., 2., 3.]], [[7., 8., 9.]]] # a rank 3 tensor with shape [2, 1, 3]

### 3.Tensorflow程序的核心流程
你或许认为Tensorflow的核心程序包括两个独立的部分：
* 1.建立计算图（tf.Graph）
* 2.运行计算图（tf.Session）

#### 3.1.Graph
一个计算图是指一系列的Tensorflow操作中包装在一个图中，Graph主要有两个部分组成：
* <a href = "">Operation</a>(或者ops)：Graph的节点，Operation描述了使用和产生Tensor的操作
* Tensor：Graph的边，他们表示了数据在Graph的工作流，大多数Tensorflow函数都会返回tf.Tensors

Note:tf.Tensor本身并不包含数值，它们只是计算图中各个元素的<a href="https://www.cnblogs.com/idorax/p/6414007.html">handle</a>

我们来建立一个简单的计算图，最基础的操作是设置constant.

In [3]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0) # also tf.float32 implicitly
total = a + b
print(a)
print(b)
print(total)

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


注意打印出的tensor并没有输出3.0,4.0,7.0,以上部分只是建立了计算图，这些Tensor对象仅仅表示了将要被执行的操作结果。

Graph中每一个操作都被赋予独一无二的名字，这个名字独立于那些在部署给python的对象的名字。Tensor在operation产生后命名，例如上面输出部分的“add:0”

#### 3.2. TensorBoard
Tensorflow提供了TensorBoard，它的主要功能是可视化计算图，你只需用几个简单的指令就可以做到。

* 首先，你需要将计算图保存到一个TensorBoard summary file中：

In [4]:
writer = tf.summary.FileWriter('.')

writer.add_graph(tf.get_default_graph())

* 该操作将在当前目录下生成event文件，文件名字的格式如下：

events.out.tfevents.{timestamp}.{hostname}

* 现在，开启一个终端，执行以下命令：

tensorboard --logdir .

在你的浏览器下打开TensorBoard的网址，大概会看到类似于以下的图：
<img src="https://www.tensorflow.org/images/getting_started_add.png?hl=zh-cn">

关于TensorBoard的更多内容，查看<a href = “”>TensorBoard的Tutorial</a>

#### 3.3 Session
每个Session包含Tensorflow运行时的状态，并且负责执行Tensorflow的operation。打个比方来说，如果tf.Graph是一个.py文件的话，那么Session就是python的执行部分

以下代码创建了一个tf.Session对象并且唤起它的run方法，来估测我们刚刚创建的名为total的Tensor。

In [5]:
sess = tf.Session()
print(sess.run(total))

7.0


当你调用Session.run请求输出一个节点时，Tensorflow将追溯图中所有提供所请求输出的输入，所以最终的执行结果是7

你也可以向Session.run()中以字典的形式传入多个tensor

In [6]:
print(sess.run({'ab':(a, b), 'total':total}))

{'ab': (3.0, 4.0), 'total': 7.0}


以及，需要注意的是，如果使用tf.random_uniform创建tensor,每次Session.run过程中都会产生不同值：

In [7]:
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1, out2)))

[0.03883779 0.82260215 0.79136217]
[0.68874    0.42780805 0.7759861 ]
(array([1.8903666, 1.4699574, 1.6417528], dtype=float32), array([2.8903666, 2.4699574, 2.6417527], dtype=float32))


#### 3.4 Feeding
如果tensorflow真的只有以上功能的话（产生定值），就太无聊啦。
<br>事实上，一个图可以接受其他输出途径，例如tf.placeholder(),它允许我们稍后再赋予具体的值，就像一个函数的参数

In [8]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y
print(z)

Tensor("add_3:0", dtype=float32)


随后，在执行，Session.run()时，我们使用feed_dict参数给placeholder喂入具体的值

In [9]:
print(sess.run(z, feed_dict={x: 3, y: 4.5}))
print(sess.run(z, feed_dict={x: [1, 3], y: [2, 4]}))

7.5
[3. 7.]


#### Note: feed_dict可以用于重写Graph中的任何tensor，但placeholder与其他tensor唯一的区别是：如果placeholder没有被赋予值会抛出错误异常

### 4. Dataset
placeholder在简单的实验下比较有效，但是在向模型输入数据流时，Dataset更值得推荐。

为了从Dataset中得到可执行的tf.Tensor,首先你需要将它转化为tf.data.Iterator,之后调用Iterator的get_next方法。

创建Iterator最简单的方式就是用make_ont_shot_iterator方法，例如，以下代码：

In [10]:
my_data = [
    [0, 1,],
    [2, 3,],
    [4, 5,],
    [6, 7,],
]
slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

next_item每次运行时都会返回my_data中的一行数据

In [11]:
while True:
    try:
        print(sess.run(next_item))
    except tf.errors.OutOfRangeError:
        break

[0 1]
[2 3]
[4 5]
[6 7]


关于Dataset的更多内容请查看<a href=“”>Importing Data的tutorial</a>

### 5.Layers
（我觉得layer是什么大家都懂...

#### 5.1 创建Layer
例如创建一个全连接层，unit是指为每一个传入的vector创建输出的数量：

In [12]:
x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)

#### 5.2 初始化Layer
包含变量的Layer在使用之前必须初始化，但通过以下简单的代码就可以实现初始化Graph中所有变量：

In [13]:
init = tf.global_variables_initializer()
sess.run(init)

#### Note：
* global_variables_initializer仅仅创建并返回tensorflow operation的handle，真正的初始化在我们执行Session.run()时完成
* global_variables_initializer仅仅初始化当它被创建时已经存在于图中的变量，所以initializer必须是创建图过程中最后一个被创建的对象

#### 5.3 执行

In [14]:
print(sess.run(y, {x: [[1, 2, 3],[4, 5, 6]]}))

[[1.5839438]
 [5.915743 ]]


如上所示，layer为每个输入的vector生成一个输出

#### 5.4 Layer函数的简便版
对于每个layer class，Tensorflow都为其创建了简便版，例如tf.layers.Dense的简便版是tf.layers.dense,区别如下：

In [15]:
x = tf.placeholder(tf.float32, shape=[None, 3])
###
y = tf.layers.dense(x, units=1)

init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))

[[ -5.5061736]
 [-12.189417 ]]


但缺点是debug过程更麻烦，且无法重复使用某次创建的Layer

### 6.Feature Columns
使用Feature Columns最简单的方法是调用tf.feature_column.input_layer函数，但该函数仅接受Dense column，所以如果想查看categorical Column，必须要将其包装在indicator column后再传入。
(关于dense/categorical column是什么，参考下图)
<img src = "https://www.tensorflow.org/images/feature_columns/some_constructors.jpg?hl=zh-cn">
（其实这些在<a href = "">Feature Column的tutorial</a>中讲的比较详细）

In [16]:
features = {
    'sales' : [[5], [10], [8], [9]],
    'department': ['sports', 'sports', 'gardening', 'gardening']}

department_column = tf.feature_column.categorical_column_with_vocabulary_list(
        'department', ['sports', 'gardening'])
department_column = tf.feature_column.indicator_column(department_column)

columns = [
    tf.feature_column.numeric_column('sales'),
    department_column
]

inputs = tf.feature_column.input_layer(features, columns)

运行inputs tensor将会把Feature分割为一批vectors

特征列(Feature Column)如同layer一样，有自己的内部结构和状态，所以通常来说需要初始化，Categorical Column在内部中使用查询表(<a href = "https://www.tensorflow.org/api_docs/python/tf/contrib/lookup?hl=zh-cn">lookup table</a>)，需要额外的初始化过程（<a href="https://www.tensorflow.org/api_docs/python/tf/tables_initializer?hl=zh-cn">tf.tables_initializer</a>）

In [18]:
var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
sess = tf.Session()
sess.run((var_init, table_init))
print(sess.run(inputs))

[[ 1.  0.  5.]
 [ 1.  0. 10.]
 [ 0.  1.  8.]
 [ 0.  1.  9.]]


### 7.Training
现在你已经熟悉Tensorflow中基本的核心概念了，现在我们来手动运行一个简单的回归模型

#### Define
首先我们来定义一些输入，以及期待的输出

In [19]:
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

#### Define the Model
随后，我们定义一个仅有一个输出的线性模型：

In [21]:
linear_model = tf.layers.Dense(units=1)

y_pred = linear_model(x)

我们先查看一下训练前的模型预测情况

In [22]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y_pred))

[[-0.8881192]
 [-1.7762384]
 [-2.6643577]
 [-3.552477 ]]


#### loss
为了优化模型我们来定义模型的loss，这里我们用回归问题的标准损失函数：平方差

<a href="https://www.tensorflow.org/api_docs/python/tf/losses?hl=zh-cn">tf.losses</a>提供了一系列常用的损失函数

In [24]:
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

print(sess.run(loss))

0.5344759


#### Training
tf.train.Optimizer提供一系列标准的优化算法. 它们通过降低损失不断改进每一个变量，最简单的优化算法是梯度下降法

In [25]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

目前这个代码已经建立了优化graph中所有必备的组件，并且返回一个可训练的operation，当运行时，opt会不断更新图中的变量：

In [27]:
for i in range(100):
    _, loss_value = sess.run((train, loss))
    print(loss_value)

0.079372294
0.078897715
0.078426
0.077957116
0.077491015
0.07702772
0.07656719
0.07610942
0.07565437
0.075202025
0.07475242
0.07430547
0.07386122
0.073419616
0.07298063
0.07254429
0.072110556
0.071679436
0.071250856
0.07082486
0.07040142
0.06998052
0.06956211
0.069146186
0.06873278
0.06832184
0.06791337
0.0675073
0.0671037
0.066702515
0.0663037
0.065907285
0.06551322
0.06512153
0.06473218
0.064345166
0.063960455
0.06357804
0.063197926
0.062820084
0.062444475
0.06207112
0.061700035
0.061331116
0.060964435
0.060599938
0.06023762
0.05987746
0.059519473
0.059163626
0.058809903
0.058458254
0.058108773
0.057761353
0.057415977
0.057072707
0.056731477
0.0563923
0.056055117
0.055719968
0.055386834
0.055055693
0.054726537
0.054399323
0.05407409
0.053750776
0.05342941
0.05310996
0.05279244
0.052476794
0.052163057
0.051851157
0.051541146
0.051233
0.050926693
0.050622195
0.050319545
0.050018698
0.049719647
0.049422372
0.04912688
0.048833158
0.048541192
0.048250973
0.04796247
0.04767572
0.047390677


#### Note：注意我们同时将loss放入run中是为了查看输出，因为train只是操作，并不返回具体值

#### Prediction